In [20]:
import numpy as np
import pandas as pd
import pybaseball

In [22]:
pybaseball.cache.enable()
pd.set_option("display.max_columns", None)

In [16]:
 pitching = pd.read_csv("data/lahman_1871-2024_csv/Pitching.csv")
 players = pd.read_csv("data/lahman_1871-2024_csv/People.csv")
 batting = pd.read_csv("data/lahman_1871-2024_csv/Batting.csv")
 appearances = pd.read_csv("data/lahman_1871-2024_csv/Appearances.csv")

NameError: name 'pybaseball' is not defined

In [17]:
pitchData = statcast(start_dt="2024-03-01", end_dt="2025-10-31")

This is a large query, it may take a moment to complete


c:\users\17083\appdata\local\programs\python\python39\lib\site-packages\pybaseball\statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates


100%|████████████████████████████████████████████████████████████████████████████████| 477/477 [06:45<00:00,  1.18it/s]


,pitch_type,release_speed,pfx_x,pfx_z,release_spin_rate
131,SL,88.0,-0.4,0.61,2619
134,SL,86.8,0.34,0.38,2635
140,SL,87.4,-0.06,0.48,2574
145,SL,86.4,0.27,0.24,2614
154,SL,86.1,0.2,0.44,2550
...,...,...,...,...,...
1666,NaN,<NA>,<NA>,<NA>,<NA>
1804,NaN,<NA>,<NA>,<NA>,<NA>
1849,NaN,<NA>,<NA>,<NA>,<NA>
2030,NaN,<NA>,<NA>,<NA>,<NA>


In [29]:
# Look at pitch movement columns
mainDF = pitchData[['game_date','player_name','events','description','p_throws','hit_location','bb_type','balls','strikes','pitch_type', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate','plate_x','plate_z','on_3b','on_2b','on_1b','outs_when_up','inning','inning_topbot','release_pos_y','at_bat_number','pitch_number','pitch_name','bat_score','fld_score','post_bat_score','post_fld_score', 'delta_home_win_exp','delta_run_exp','bat_speed','swing_length','pitcher_days_since_prev_game','n_thruorder_pitcher', 'arm_angle', 'batter','pitcher']]

In [43]:
ver = mainDF[mainDF['player_name'] == 'Verlander, Justin'].copy()

In [47]:
ver["events"].value_counts()

field_out                    470
strikeout                    221
single                       173
walk                          81
double                        58
home_run                      31
hit_by_pitch                  12
sac_fly                       12
force_out                     12
grounded_into_double_play     11
field_error                    8
sac_bunt                       3
triple                         3
truncated_pa                   3
fielders_choice                2
double_play                    2
fielders_choice_out            2
Name: events, dtype: int64

In [57]:
ver["pitch_success"] = np.where(
    (
        ver["events"].isin([
            'field_out', 'strikeout', 'field_error', 'fielders_choice',
            'double_play', 'grounded_into_double_play', 'force_out'
        ])
    ) |
    (
        ver["description"].isin(['called_strike', 'swinging_strike'])
    ) |
    (
        ver["description"].isin(['foul']) & (ver['strikes'] != 2)
    ),
    True,
    False
)

In [58]:
conditions = [
    ver["events"].isin(['double_play', 'grounded_into_double_play']),
    ver["events"].isin(['strikeout']), 
    ver["events"].isin(['field_out']),
    ver["description"].isin(['swinging_strike', 'called_strike']),
    ver["description"].isin(['foul', 'foul_tip']),
    ver["description"].isin(['ball']),
    ver["description"].isin(['hit_by_pitch']),
    ver["events"].isin(['single']),  # bad outcomes
    ver["events"].isin(['double']),  # bad outcomes
    ver["events"].isin(['triple']),  # bad outcomes
    ver["events"].isin(['home_run']),  # bad outcomes
]

values = [1.00, 0.95, 0.90, 0.75, 0.70, 0.50, 0.25, 0.25, 0.20, 0.15, 0]

ver["pitch_outcome_score"] = np.select(conditions, values, default=np.nan)

In [59]:
ver

,game_date,player_name,events,description,p_throws,hit_location,bb_type,balls,strikes,pitch_type,release_speed,pfx_x,pfx_z,release_spin_rate,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,release_pos_y,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,post_bat_score,post_fld_score,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,pitcher_days_since_prev_game,n_thruorder_pitcher,arm_angle,batter,pitcher,pitch_success,pitch_outcome_score
28,2025-09-27,"Verlander, Justin",field_out,hit_into_play,R,4,line_drive,2,2,ST,81.1,0.84,0.0,2660,-0.29,1.62,<NA>,<NA>,<NA>,2,6,Top,54.7,42,5,Sweeper,2,3,2,3,0.017,-0.224,69.4,8.3,5,3,51.3,692585,434378,True,0.90
36,2025-09-27,"Verlander, Justin",NaN,foul,R,<NA>,NaN,2,1,CU,78.1,0.74,-0.95,2760,0.87,1.94,<NA>,<NA>,<NA>,2,6,Top,54.55,42,4,Curveball,2,3,2,3,0.0,-0.073,76.4,8.4,5,3,63.7,692585,434378,True,0.70
50,2025-09-27,"Verlander, Justin",NaN,swinging_strike,R,<NA>,NaN,2,0,CU,77.6,0.65,-0.95,2715,0.71,1.04,<NA>,<NA>,<NA>,2,6,Top,54.67,42,3,Curveball,2,3,2,3,0.0,-0.065,69.7,9.1,5,3,56.1,692585,434378,True,0.75
54,2025-09-27,"Verlander, Justin",NaN,ball,R,<NA>,NaN,1,0,CU,77.7,0.64,-0.98,2785,-1.27,3.26,<NA>,<NA>,<NA>,2,6,Top,54.78,42,2,Curveball,2,3,2,3,0.0,0.065,<NA>,<NA>,5,3,61.3,692585,434378,False,0.50
60,2025-09-27,"Verlander, Justin",NaN,ball,R,<NA>,NaN,0,0,FF,93.7,-0.69,1.61,2490,-0.54,4.89,<NA>,<NA>,<NA>,2,6,Top,54.67,42,1,4-Seam Fastball,2,3,2,3,0.0,0.037,<NA>,<NA>,5,3,63.7,692585,434378,False,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3605,2024-04-19,"Verlander, Justin",NaN,ball,R,<NA>,NaN,0,1,CU,76.9,0.79,-1.04,2585,-1.26,3.15,<NA>,<NA>,<NA>,1,1,Bot,54.93,5,2,Curveball,0,0,0,0,0.0,0.024,<NA>,<NA>,<NA>,1,55.6,682928,434378,False,0.50
3679,2024-04-19,"Verlander, Justin",NaN,foul,R,<NA>,NaN,0,0,FF,93.5,-0.8,1.7,2474,-0.16,2.66,<NA>,<NA>,<NA>,1,1,Bot,54.59,5,1,4-Seam Fastball,0,0,0,0,0.0,-0.037,76.8,7.3,<NA>,1,58.0,682928,434378,True,0.70
3819,2024-04-19,"Verlander, Justin",field_out,hit_into_play,R,1,ground_ball,1,1,SL,87.0,0.55,0.72,2578,0.85,2.01,<NA>,<NA>,<NA>,0,1,Bot,54.49,4,3,Slider,0,0,0,0,-0.021,-0.243,68.1,7.0,<NA>,1,52.0,696285,434378,True,0.90
3997,2024-04-19,"Verlander, Justin",NaN,called_strike,R,<NA>,NaN,1,0,FF,93.0,-0.74,1.55,2470,0.32,2.94,<NA>,<NA>,<NA>,0,1,Bot,54.48,4,2,4-Seam Fastball,0,0,0,0,0.0,-0.049,<NA>,<NA>,<NA>,1,54.4,696285,434378,True,0.75
